# 初始設定

In [ ]:
%%capture
!pip install transformers
!pip install gdown
!pip install opencc-python-reimplemented
!pip install googletrans==3.1.0a0
!pip install fake_useragent

In [ ]:
import torch
import numpy as np
import pandas as pd
import re

In [ ]:
from transformers import BertModel, BertTokenizer
PATH = '/content/FinBERT_L-12_H-768_A-12_pytorch'

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent


In [ ]:
import time
import datetime as dt
import time
import tqdm

In [ ]:
from opencc import OpenCC
cc = OpenCC('t2s')

In [ ]:
#!gdown https://drive.google.com/uc?id=1qW1YWtw3q9Q28QThrIY-rDU9Gl-SLIKO
#!unzip /content/FinBERT_L-12_H-768_A-12_pytorch.zip

# 財金BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PATH ,local_files_only=True)
model = BertModel.from_pretrained(PATH ,local_files_only=True)

In [ ]:
sector_dict = {
'健護': '醫療保健服務類股',
'科技': '科技類股', 
'公用事業': '公用事業類股',
'生技':'生技類股', 
'能源':'能源類股',
'替代能源':'替代能源類股',
'消費':'消費品類股',
'金融':'金融類股', 
'礦業':'礦業',
'天然資源':'天然資源', 
'貴金屬現貨':'貴金屬',
'農產品':'農產品',
'貴金屬類股':'貴金屬',
}

In [ ]:
mkt_dict= [
'新興市場債',
'高收益債',
'全球債',
'投資等級公司債',
'美國公債',
'亞債',
'REITs不動產投資信託',
'新臺幣類貨幣',
'外幣類貨幣',
'亞太股市', '美國股市', '全球新興市場', 
'全球型股市', '大中華股市', '中國A股', '印度股市','歐洲股市', 
'東南亞股市', '拉丁美洲股市', 
'韓國股市', '新興歐洲股市', '德國股市', '台灣股市','俄羅斯股市',
'中國H股', '日本股市', '英國股市','泰國股市',
'香港股市', '澳洲股市', '印尼股市', '巴西股市', '歐非中東股市'
]


In [ ]:
mkt_dict = {x:x for x in mkt_dict}
mkt_dict['亞債'] = '亞洲債券'
mkt_dict['REITs不動產投資信託'] = '不動產'

In [ ]:
total_sec_dict = {**sector_dict, **mkt_dict}

In [ ]:
sentences = list(set(total_sec_dict.values()))
sentences = [cc.convert(x) for x in sentences]

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']
np.savetxt('bert_mkt_emb.tsv', x[:,0].cpu().detach().numpy(), delimiter="\t")
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

# 鉅亨網爬蟲 初步整理


設定UserAgent

In [ ]:
ua = UserAgent()
headers = {'user-agent': ua.random}

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [ ]:
headers

{'user-agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; Media Center PC 6.0; InfoPath.2; MS-RTC LM 8)'}

## 原始方法 
直接從頁面爬取

In [ ]:
res = requests.get('https://news.cnyes.com/news/cat/wd_stock')
soup = BeautifulSoup(res.text, 'html.parser')


news_list = soup.find_all("a", attrs={"class":"_1Zdp"})
[x['href'] for x in news_list]
[x.find('h3').get_text(strip=True) for x in news_list]

## .get_text(strip=True) 可避免不必用空白符亂碼

['德外送巨頭Delivery Hero縮減Foodpanda德國業務、出售日本事業',
 '鉅亨美股雷達：拜登呼籲美國人接種加強針 馬斯克稱已賣足夠的特斯拉股票',
 '奇景光電推新一代車用TCON 明年CES亮相',
 '拓展陸地物流版圖 馬士基斥資36億美元收購香港利豐物流',
 '中國有條件批准SK海力士收購英特爾NAND業務',
 '大馬暴雨釀災 Panasonic和多家日系車廠被迫停工',
 '泰國央行連13次維持利率不變 關注Omicron變種病毒威脅',
 '日本首度傳出Omicron社區感染 共4人確診',
 '馬斯克稱已經賣了「足夠多」股票 抨擊加州稅收過高',
 '美國公司第3季庫藏規模創新紀錄 資本支出仍少於疫情前',
 '全球最大離岸風電場正式運作 已經首度發電',
 '〈2022美股展望〉美股僅剩五大科技巨頭撐場 明年多頭現疲軟跡象',
 '檢調：波音義大利供應商出包 瑕疵零件逾4000個',
 'OLED電視累計銷量突破2000萬台 LGD相關部門可望首度轉虧為盈',
 '〈2022美股展望〉華爾街明年怎麼看美股？三大重點一次掌握',
 '供應瓶頸不是重點 Nike股價暗示明年消費力道才是市場焦點',
 '紐約市防範Omicron擴散 打加強劑可拿100美元',
 '特斯拉利空逐漸鈍化 四大催化劑有望帶領多頭重拾信心',
 'AT&T甩燙手山芋 微軟買下Xandr程序化廣告公司',
 '抗疫將迎里程碑 FDA傳本周批准Merck、輝瑞口服藥',
 '美股重點新聞摘要2021年12月22日',
 'AZ傳與牛津大學開始合作開發抗Omicron疫苗',
 '蘋果加入3A頂級信評俱樂部 穆迪看好流動性和獲利前景',
 '擔憂Omicron疫情 Meta、推特、Pinterest宣布不參加明年CES',
 '拜登與世衛不同調 美CDC稱兩劑疫苗可能不夠',
 '〈美股熱門股〉軟體公司Citrix Systems漲逾13% 傳Elliott和Vista競購',
 '食物中毒產線停滯 路透：鴻海印度清奈廠本周將繼續關閉',
 '〈美股盤後〉美光瘋漲逾10% 道瓊收紅逾560點',
 '調查：蘋果iPhone稱霸全球5G市場 三星超車OPPO',
 '〈美股熱門股〉美光財報、財測俱佳 分析師齊唱旺 盤中漲逾10%']

## 將資料起訖日轉換成unix timestamp

一種時間格式，從UTC1970年1月1日0時0分0秒起至現在的總秒數


In [ ]:
dt.date(2021,1,1).timetuple()

time.struct_time(tm_year=2021, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=1, tm_isdst=-1)

In [ ]:
time.mktime(dt.date(2021,1,1).timetuple())

1609459200.0

In [ ]:
## 將資料起迄日轉換unix timestamp
## 從20210101 - 20211115
start_unix_timestamp = int(time.mktime(dt.date(2021,1,1).timetuple()))
end_unix_timestamp = int(time.mktime(dt.date(2021,11,15).timetuple()))

## 直接爬內部API 
方便，又很多東西可以挖

格式化字串(format string) 小教學

In [ ]:
print('hi, {}'.format('Hello!'))

print('hi, I am {name}'.format(name = 'Alex'))

hi, Hello!
hi, I am Alex


將起訖日參數填入，爬取第一頁
從回傳的資料以此先得知總共有幾頁要爬

In [ ]:
url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(
    start_unix_timestamp, 
    end_unix_timestamp, 
    1
)
res = requests.get(url_api, headers=headers)
res_json = res.json()

In [ ]:
url_api

'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt=1609459200&endAt=1636934400&limit=30&page=1'

In [ ]:
last_page = res_json['items']['last_page']
last_page

613

開始把613頁全爬完

In [ ]:
news_data_list = []

for i in tqdm.tqdm(range(1,last_page+1)):
  url_api = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt={}&endAt={}&limit=30&page={}'.format(start_unix_timestamp, end_unix_timestamp, i)
  res = requests.get(url_api, headers=headers)
  res_json = res.json()
  news_data_list.append(res_json)
  time.sleep(1)

100%|██████████| 613/613 [23:20<00:00,  2.29s/it]


寫入pickle

In [ ]:
import pickle
'''
with open('鉅亨網2021新聞.pickle', 'wb') as f:
  pickle.dump(news_data_list, f)
'''

"\nwith open('鉅亨網2021新聞.pickle', 'wb') as f:\n  pickle.dump(news_data_list, f)\n"

# 資料讀入

In [ ]:
import pickle

In [ ]:
!gdown https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws
with open('/content/___2021__.pickle', 'rb') as f:
  news_data_list = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?id=1ATL1IdoLgLfV-ro1r5BQzPp8DZXVqfws
To: /content/___2021__.pickle
100% 61.3M/61.3M [00:00<00:00, 109MB/s] 


## 前處理函數

正則表達式:  只保留英文中文數字 


In [ ]:
def remove_punctuation(line):
  rule = re.compile("[^a-z A-Z \u4e00-\u9fa5]")
  #rule = re.compile("[^a-z A-Z 0-9 \u4e00-\u9fa5]")
  line = rule.sub('',line)
  return line


正則表達式: 去除亂碼


In [ ]:
## 去除亂碼
def remove_char(line):
  rule = re.compile('&lt;.+?(&gt;|>)')
  line = rule.sub('',line)
  return line


UNIX timestamp轉日期

In [ ]:
from datetime import datetime
ts = int('1636969777')

## UNIX timestamp轉日期
print(dt.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-11-15 09:49:37


從Json中出取資訊

- content: 本文
- summary: 新聞摘要
- tittle: 新聞標題
- market: 新聞提到的標的


In [ ]:
news_data_flatten = []
for sublist in news_data_list:
    new_dict_list = [dict(x) for x in sublist['items']['data']]
    news_data_flatten.extend(new_dict_list)

In [ ]:
news_data_flatten[0:2]

[{'abTesting': None,
  'categoryId': 827,
  'categoryName': '台股新聞',
  'columnists': None,
  'content': '&lt;p&gt;混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元，預計第四季至明年第一季財報入帳。&lt;/p&gt;\n\n&lt;p&gt;國產這次售出的「悅誠廣場」土地資產，土地面積 4053 坪，為地上 3 樓、地下 2 樓的建物，總樓地板面積約 1.2 萬坪，由國產與其他地主共同持分，2016 年由興富發向國產等地主承租。&lt;/p&gt;\n\n&lt;p&gt;國產強調，目前全台仍擁有 45.7 萬坪的資產，會配合整體經濟現勢與區域發展趨勢，借力於未來重大交通建設與都市更新相關的商圈規劃，發展出綜合商場、渡假旅館 / 國際精品飯店、多功能辦公大樓、休閒農場等多項主題事業。&lt;/p&gt;\n\n&lt;p&gt;國產董事會日前也通過 158 億餘元的投資預算，因應「南港大型開發案」相關營造發包等作業，該案佔地 6500 坪，預估開發總樓地板面積約 6 萬餘坪，興建完成後，國產可分回 4.6 萬建坪及約 550 個停車位。&lt;/p&gt;\n',
  'coverSrc': {'l': {'height': 359,
    'src': 'https://cimg.cnyes.cool/prod/news/4771793/l/7e7c47f106072253eb6e53271586570d.jpg',
    'width': 640},
   'm': {'height': 213,
    'src': 'https://cimg.cnyes.cool/prod/news/4771793/m/7e7c47f106072253eb6e53271586570d.jpg',
    'width': 380},
   's': {'height': 101,
    'src': 'https://cimg.cnyes.cool/prod/news/4771793/s/7e7

In [ ]:
from collections import defaultdict

轉換為dataframe

In [ ]:
def def_value():
  return [{'name':None}]

In [ ]:
news_extracted_list=[]

for news in news_data_flatten:
  x = defaultdict(def_value,news)
  
  data_dict = [{
      'id':x['newsId'], 
      'title':x['title'], 
      'summary':x['summary'],  
      'article':remove_char(x['content']),  
      'stock':[x['name'] for x in x['market']],

      'unix_timestamp':x['publishAt'],
      'datetime':dt.datetime.utcfromtimestamp(x['publishAt']).strftime('%Y-%m-%d')
      } 
  ]
  news_extracted_list.extend(data_dict)


In [ ]:
org_corpus_df = pd.DataFrame(news_extracted_list)

In [ ]:
corp_list = []
for sublist in org_corpus_df['stock'].tolist():
    corp_list.extend(sublist)

In [ ]:
corp_list = list(filter(None,list(set(corp_list))))

去除換行符號與空白

In [ ]:
org_corpus_df['article'] = org_corpus_df['article'].str.replace('\n',"")
org_corpus_df['article'] = org_corpus_df['article'].str.replace(" ", "")

In [ ]:
org_corpus_df.head()

,id,title,summary,article,stock,unix_timestamp,datetime
0,4771793,國產出售高雄悅誠廣場資產 貢獻EPS 1.27元,混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 2...,混凝土廠國產(2504-TW)今(15)日公告，將高雄「悅誠廣場」房地持分以總價23.5億元...,"[國產, 興富發]",1636984978,2021-11-15
1,4771790,〈台泥法說〉Q4煤價下滑成本壓力減 毛利率將回升至平均之上,台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙...,台泥(1101-TW)今(15)日召開法說，總經理李鐘培表示，10月以來中國能耗雙控持續，水...,[台泥],1636984833,2021-11-15
2,4771785,〈台泥法說〉張安平談氣候變遷「水泥業站在生命第一現場」 承擔減碳責無旁貸,台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，...,台泥(1101-TW)董事長張安平今(15)日在法說會前致詞時指出，今天的氣候災害，是人類打...,[台泥],1636984795,2021-11-15
3,4771771,達邁調整產品組合 看好高頻高速傳輸及汽車市場應用,聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 9...,聚醯亞胺（PI）薄膜廠達邁(3645-TW)積極展開產品多元化布局，以往高達98%的軟板應用...,[達邁],1636980649,2021-11-15
4,4771770,22家壽險前3季合賺3389億元破紀錄 淨值比all pass！,22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！,壽險公司第3季財報全數公布，22家壽險公司合計前3季稅後純益逾3389億元，創史上新高，年增...,[None],1636980334,2021-11-15


# 進行分詞

轉成list格式以利分詞

先取用摘要進行探勘

In [ ]:
text = org_corpus_df['summary'].astype(str).tolist()

In [ ]:
text[0:5]

['混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元。',
 '台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙控持續，水泥產銷量減但價格持穩高檔，且近期煤價大幅回落，有利毛利率回升到過往水準之上，加上廣東韶關廠投產也將挹注營收表現。',
 '台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，是人類打造繁榮的副作用漸漸浮現、失衡的大自然對人類的反撲；與人類文明與大自然息息相關的水泥業「站在生命的第一現場」。',
 '聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 98% 的軟板應用，在今年前三季的比重已降至 64%，而在非軟板的應用的成長到 36% 則以車用、散熱產品為主。',
 '22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！']

### 載入停用詞

載入中文之中「的、了、與其、和」這種意義不大詞彙

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt

In [ ]:
## 逐行讀入
with open('/content/cn_stopwords.txt', 'r') as f:
  stop_words = f.read().splitlines()

使用opencc套件進行簡轉繁

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2t')

## 轉繁體
stop_words = [cc.convert(text) for text in stop_words]

In [ ]:
## 保留數字
stop_words = list(filter(lambda x: x not in ['0','1','2','3','4','5','6','7','8','9'], stop_words))

In [ ]:
stop_words.extend(['TW','tw'])

In [ ]:
stop_words[100:110]

['介於', '仍', '仍舊', '從', '從此', '從而', '他', '他人', '他們', '以']

## 結巴jieba分詞

最知名的中文分詞套件，繁中效果比中研院斷詞差一些，但很多套件會用這個做為基礎，還是得要會

- [加入自定義字典方法](https://github.com/fxsjy/jieba/blob/master/test/test_userdict.py) 

In [ ]:
import jieba

In [ ]:
%%time
corpus_ws_jb_org = [list(jieba.cut(x, cut_all=False)) for x in text]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.031 seconds.
Prefix dict has been built successfully.


CPU times: user 15 s, sys: 187 ms, total: 15.2 s
Wall time: 16.3 s


In [ ]:
corpus_ws_jb_org[0:2]

[['混凝土',
  '廠國產',
  ' ',
  '(',
  '2504',
  '-',
  'TW',
  ')',
  ' ',
  '今',
  ' ',
  '(',
  '15',
  ')',
  ' ',
  '日',
  '公告',
  '，',
  '將高雄',
  '「',
  '悅誠廣場',
  '」',
  '房地持',
  '分以',
  '總價',
  ' ',
  '23.5',
  ' ',
  '億元',
  '，',
  '出售',
  '予興',
  '富發',
  ' ',
  '(',
  '2542',
  '-',
  'TW',
  ')',
  ' ',
  '子公司',
  '碧江',
  '企業',
  '，',
  '處分',
  '利益',
  '達',
  ' ',
  '14.9',
  ' ',
  '億元',
  '，',
  '估貢獻',
  '每股',
  '純益',
  '約',
  ' ',
  '1.27',
  ' ',
  '元',
  '。'],
 ['台泥',
  ' ',
  '(',
  '1101',
  '-',
  'TW',
  ')',
  ' ',
  '今',
  ' ',
  '(',
  '15',
  ')',
  ' ',
  '日召',
  '開法',
  '說',
  '，',
  '總經理',
  '李鐘培',
  '表示',
  '，',
  '10',
  ' ',
  '月',
  '以來',
  '中國',
  '能耗',
  '雙控',
  '持續',
  '，',
  '水泥',
  '產銷量',
  '減但',
  '價格持',
  '穩高檔',
  '，',
  '且',
  '近期',
  '煤價',
  '大幅',
  '回落',
  '，',
  '有利',
  '毛利率',
  '回升',
  '到',
  '過往',
  '水準',
  '之上',
  '，',
  '加上',
  '廣東韶關',
  '廠',
  '投產',
  '也將',
  '挹注',
  '營收表現',
  '。']]

## 中研院開發BERT版NLP套件

有分詞、實體辨識、詞性判斷等功能，效果提升但比較慢

In [ ]:
%%capture
"""
!pip install kora
from kora.xattr import get_id
fid = get_id('/content/drive/MyDrive/03_RESOURCE/鉅亨網_ws.pickle')
"""

In [ ]:
!gdown https://drive.google.com/uc?id=1kQ5EhInxn4xO-rdZQ5VfwYpI5ejFBPOq
with open('/content/____ws.pickle', 'rb') as f:
  corpus_ws_org = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?id=1kQ5EhInxn4xO-rdZQ5VfwYpI5ejFBPOq
To: /content/____ws.pickle
100% 9.85M/9.85M [00:00<00:00, 86.8MB/s]


In [ ]:
%%capture
!pip install -U ckip-transformers
#https://github.com/ckiplab/ckip-transformers


In [ ]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

- `level` 1-3: 3準確度高速度最慢
- `device=0`: 採用GPU

In [ ]:
## 設定driver
ws_driver = CkipWordSegmenter(level=3,device=0)

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/388M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

開始進行分詞

In [ ]:
%%time
corpus_ws_org  = ws_driver(text)

Inference: 100%|██████████| 66/66 [04:18<00:00,  3.91s/it]


CPU times: user 4min 23s, sys: 982 ms, total: 4min 24s
Wall time: 4min 26s


In [ ]:
corpus_ws_org[0]

['混凝土廠',
 '國產',
 ' ',
 '(',
 '2504-TW',
 ')',
 ' ',
 '今',
 ' ',
 '(15) ',
 '日',
 '公告',
 '，',
 '將',
 '高雄',
 '「',
 '悅誠',
 '廣場',
 '」',
 '房地',
 '持分',
 '以',
 '總價',
 ' 23.5 億',
 '元',
 '，',
 '出售予',
 '興富發',
 ' ',
 '(',
 '2542-TW',
 ')',
 ' ',
 '子公司',
 '碧江',
 '企業',
 '，',
 '處分',
 '利益',
 '達',
 ' 14.9 億',
 '元',
 '，',
 '估',
 '貢獻',
 '每',
 '股',
 '純益',
 '約',
 ' 1.27',
 ' ',
 '元',
 '。']

In [ ]:
'''
with open('鉅亨網_ws.pickle', 'wb') as f:
  pickle.dump(corpus_ws_org, f)
!cp /content/鉅亨網_ws.pickle /content/drive/MyDrive/03_RESOURCE
'''

## 中研院Bi-LSTM斷詞

先前推出的，速度較快

In [ ]:
!pip install -U ckiptagger[gdown] -q

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
!gdown https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
!unzip data.zip

ws = WS("./data", disable_cuda=False)

In [ ]:
%%time
corpus_ws_org  = ws(text)

CPU times: user 56.9 s, sys: 6.75 s, total: 1min 3s
Wall time: 52.9 s


## 將處理完的分詞進一步進行清理（停用詞、符號等）

todo
- 詞性判斷
- 股票dict



In [ ]:
corpus_ws_org[0:2]

[['混凝土廠',
  '國產',
  ' ',
  '(',
  '2504-TW',
  ')',
  ' ',
  '今',
  ' ',
  '(15)',
  ' ',
  '日',
  '公告',
  '，',
  '將',
  '高雄',
  '「',
  '悅誠',
  '廣場',
  '」',
  '房地',
  '持分',
  '以',
  '總價',
  ' ',
  '23.5',
  ' 億',
  '元',
  '，',
  '出售',
  '予',
  '興富',
  '發',
  ' ',
  '(',
  '2542',
  '-TW',
  ')',
  ' ',
  '子公司',
  '碧江',
  '企業',
  '，',
  '處分',
  '利益',
  '達',
  ' ',
  '14.9',
  ' 億',
  '元',
  '，估',
  '貢獻',
  '每',
  '股',
  '純益',
  '約',
  ' ',
  '1.27',
  ' ',
  '元',
  '。'],
 ['台泥',
  ' ',
  '(',
  '1101-TW',
  ')',
  ' ',
  '今',
  ' ',
  '(15)',
  ' ',
  '日',
  '召開',
  '法',
  '說',
  '，',
  '總經理',
  '李鐘培',
  '表示',
  '，',
  '10',
  ' ',
  '月',
  '以來',
  '中國',
  '能',
  '耗',
  '雙控',
  '持續',
  '，',
  '水泥',
  '產銷量',
  '減',
  '但',
  '價格',
  '持穩',
  '高檔',
  '，',
  '且',
  '近期',
  '煤價',
  '大幅',
  '回落',
  '，',
  '有利',
  '毛利率',
  '回升',
  '到',
  '過往',
  '水準',
  '之上',
  '，',
  '加上',
  '廣東',
  '韶關',
  '廠',
  '投產',
  '也',
  '將',
  '挹注',
  '營收',
  '表現',
  '。']]

因為是lists of lists
```
[
  [文件1分詞1, 文件1分詞2, 文件1分詞3...], 
  [文件2分詞1, 文件2分詞2, 文件2分詞3...]
]
```
所以要包兩層處理

In [ ]:
## 刪除標點
corpus_ws = [list(map(lambda x: remove_punctuation(x), corpus)) for corpus in corpus_ws_org]

In [ ]:
## 刪除空白的element
corpus_ws = [list(filter(lambda x: x!="" and x!=" ", corpus)) for corpus in corpus_ws]

In [ ]:
## 刪除element裡的空白
pattern = re.compile(r'\s+')
corpus_ws = [list(map(lambda x: re.sub(pattern, '', x), corpus)) for corpus in corpus_ws]

In [ ]:
## 把停用詞刪除
corpus_ws = [[x for x in sentence if x not in stop_words] for sentence in corpus_ws]

In [ ]:
## 只保留兩個字以上的分詞
corpus_ws = [list(filter(lambda x: len(x)>=2, corpus)) for corpus in corpus_ws]

In [ ]:
corpus_ws[0:2]

[['混凝土廠',
  '國產',
  '公告',
  '高雄',
  '悅誠',
  '廣場',
  '房地',
  '持分',
  '總價',
  '出售',
  '興富',
  '子公司',
  '碧江',
  '企業',
  '處分',
  '利益',
  '貢獻',
  '純益'],
 ['台泥',
  '召開',
  '總經理',
  '李鐘培',
  '表示',
  '中國',
  '雙控',
  '持續',
  '水泥',
  '產銷量',
  '價格',
  '持穩',
  '高檔',
  '近期',
  '煤價',
  '大幅',
  '回落',
  '有利',
  '毛利率',
  '回升',
  '過往',
  '水準',
  '之上',
  '加上',
  '廣東',
  '韶關',
  '投產',
  '挹注',
  '營收',
  '表現']]

In [ ]:
org_corpus_df['summary_seg'] = corpus_ws

In [ ]:
org_corpus_df['summary_list_len'] = org_corpus_df['summary_seg'].apply(lambda x: len(x))

In [ ]:
org_corpus_df['summary_seg_join']  = org_corpus_df['summary_seg'].apply(lambda x: " ".join(x))

有些套件只能用raw text直接套入，並用英文的方式解析(詞跟詞用空白分隔、用.結尾)

In [ ]:
org_corpus_df['summary_seg_join'] 

0        混凝土廠 國產 公告 高雄 悅誠 廣場 房地 持分 總價 出售 興富 子公司 碧江 企業 處...
1        台泥 召開 總經理 李鐘培 表示 中國 雙控 持續 水泥 產銷量 價格 持穩 高檔 近期 煤...
2        台泥 董事長 張安平 法說會 致詞 指出 今天 氣候 災害 人類 打造 繁榮 副作用 漸漸 ...
3        聚醯亞胺 PI 薄膜廠 達邁 積極 展開 產品 應用 多元化 布局 以往 高達 軟板 應用 ...
4                                     壽險 合賺 破紀錄 淨值 allpass
                               ...                        
18364    泛國 集團 電感廠 奇力 受惠 手機 伺服器 基地台 物聯網 汽車 帶動 一體 成形 微型 ...
18365    環宇 KY 去年 中國 基礎 建設 招標 遞延 影響 加上 轉投資 晶成 半導體 常州 承芯...
18366    閎暉 近年 積極 調整 體質 過去 手機 按鍵 業務 比例 降低 聚焦 車用 穿戴 裝置 業...
18367                              潛力股 電動車 成長 順風車 獲利 動能 加溫
18368    全球 車市 需求 可望 新冠肺炎 疫苗 研發 有成 帶動 進一步 扭轉 汽車板廠 過去 淡季...
Name: summary_seg_join, Length: 18369, dtype: object

## 整合函數

In [ ]:
def ws_whole(to_ws_raw_text):
  to_ws_text = [to_ws_raw_text]
  ws_text = ws_driver(to_ws_text)[0]
  ws_text = list(map(lambda x: remove_punctuation(x), ws_text))
  ws_text = list(filter(lambda x: x!="" and x!=" ", ws_text))
  pattern = re.compile(r'\s+')
  ws_text = list(map(lambda x: re.sub(pattern, '', x), ws_text))
  ws_text = [x for x in ws_text if x not in stop_words]

  return ws_text

# Text mining

## 關鍵字萃取

### 中文TF-IDF

`token_pattern='\\b\\w+\\b'` : 讓TF-IDF認得單字詞

`lowercase=False`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

建立詞頻矩陣


In [ ]:
text_cv = CountVectorizer(max_df=0.7, min_df=5)
td_matrix = text_cv.fit_transform(org_corpus_df['summary_seg_join'])

詞頻矩陣套入TF-IDF


In [ ]:
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(td_matrix)

每篇文章挑出文章內tfidf前五名字詞

In [ ]:
topk_arr = np.argsort(-tfidf_matrix.toarray(), axis=1)[:,0:5]

In [ ]:
indices = np.nonzero(-tfidf_matrix.toarray())[0]
#indices[np.argsort(-tfidf_matrix.toarray()[indices])][:,0:5]

In [ ]:
org_corpus_df['tf_idf_kw'] = text_cv.get_feature_names_out()[topk_arr].tolist()
org_corpus_df[['summary','tf_idf_kw']].iloc[20:30,:]

,summary,tf_idf_kw
20,合庫銀私人銀行今 (15) 日正式揭牌營業，為了與市場區隔化，私人銀行經營項目將聚焦資產配置...,"[專業, 資產, 服務, 銀行, 傳承]"
21,藥華藥 (6446-TW) 今 (15) 日舉行法說，外界關注，治療真性紅血球增多症 (PV...,"[真性, 增多症, pv, 紅血球, 藥華藥]"
22,漢民集團旗下漢磊 (3707-TW) 與轉投資磊晶廠嘉晶 (3016-TW) 今 (15) ...,"[sic, 漢民, 嘉晶, gan, 產能]"
23,PMOLED 廠錸寶 (8104-TW) 今 (15) 日召開法說，展望未來，錸寶看好電池儲...,"[錸寶, pmoled, 儲能, 成長性, 自動]"
24,統包工程承攬商中鼎 (9933-TW) 集團，攜手欣陸 (3703-TW) 旗下欣達環工，執...,"[bot, 承攬商, 欣陸, 桃園市, 統包]"
25,封測龍頭日月光投控 (3711-TW) 今 (15) 日宣布，連續第六年整體成績獲道瓊永續指...,"[產業, 連續, 半導體, 第六, 成績]"
26,漢磊 (3707-TW) 今 (15) 日表示，SiC 產線第四季全數滿載，整體毛利率將持續...,"[毛利率, 漢磊, sic, 起跳, 續揚]"
27,國產署近期與高雄、屏東兩縣市地方政府合作辦理 2 件國有土地開發案，預計可吸引民間投資 60...,"[國有, 屏東, 國產署, 民間, 縣市]"
28,國銀人民幣存款連 3 減 個人戶連 8 個月下滑,"[存款, 國銀, 人民幣, 下滑, 社交]"
29,PMOLED 廠錸寶 (8104-TW) 今 (15) 日召開法說，展望第四季，錸寶表示，P...,"[錸寶, pmoled, 業務, 轉趨, 塞港]"


array([['興富', '總價', '房地', '出售', '國產'],
       ['廣東', '產銷量', '過往', '回落', '水泥'],
       ['人類', '現場', '生命', '失衡', '張安平'],
       ...,
       ['業務', '明年', '穿戴', '閎暉', '訂單量'],
       ['加溫', '電動車', '動能', '獲利', '成長'],
       ['汽車板廠', '扭轉', '頹勢', '敬鵬', '拉高']], dtype=object)

### TextRank

目前效果比較完善的套件是丟入raw text
他會自己用結巴幫你分好算出關鍵字，還可以萃取摘要句

若要套用中研院斷詞，需要研究內部原始碼

https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0

In [ ]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
from gensim.summarization import keywords

In [ ]:
". ".join(org_corpus_df[org_corpus_df['summary_list_len']>0]['summary_seg_join'][0:3])



'混凝土廠 國產 公告 高雄 悅誠 廣場 房地 持分 總價 出售 興富 子公司 碧江 企業 處分 利益 貢獻 純益. 台泥 召開 總經理 李鐘培 表示 中國 雙控 持續 水泥 產銷量 價格 持穩 高檔 近期 煤價 大幅 回落 有利 毛利率 回升 過往 水準 之上 加上 廣東 韶關 投產 挹注 營收 表現. 台泥 董事長 張安平 法說會 致詞 指出 今天 氣候 災害 人類 打造 繁榮 副作用 漸漸 浮現 失衡 大自然 人類 反撲 人類 文明 大自然 息息相關 水泥業 生命 第一 現場'

In [ ]:
text = ". ".join(org_corpus_df[org_corpus_df['summary_list_len']>0]['summary_seg_join'][0:500])


In [ ]:
keywords(text, ratio=0.2, words=None, split=False, scores=False, pos_filter=(), lemmatize=False, deacc=False).split('\n')

['董事長',
 '總經理',
 '半導體',
 '執行長',
 '台積電',
 '轉投資',
 'eps',
 '代工廠',
 '聯發科',
 '子公司',
 'led',
 '供應鏈',
 '成交量',
 '法說會',
 '電子股',
 '力積電',
 '副院長',
 '息息相關',
 '投資人',
 'llc',
 '照過來',
 '副總經理',
 'lordstown',
 'aaplus',
 '營運長',
 '財政部',
 '爆欠債',
 '毛利率',
 'flash',
 'iot',
 '英特爾',
 '矽晶圓',
 'cpi 綜所稅 免稅額',
 '雷蒙多',
 '鋼鐵人',
 '航海王',
 'gina',
 'fbus',
 'meta',
 '富邦金',
 '腦中風 退化性',
 '國泰金',
 '產品線',
 '記憶體',
 'rom',
 '年增率',
 '活蹦亂跳',
 'mosfet',
 'corporation',
 '龐貝氏症',
 '鋁合金',
 'sic',
 '智伸科',
 '裕日車',
 'tof',
 'solutions',
 '價漲量',
 'genzyme',
 '排行榜',
 '大半導體 通路廠',
 'rivian',
 'fcc',
 'rivnus',
 '鏡頭廠 玉晶光',
 'switch',
 '安侯建業',
 '任天堂',
 '研發出',
 '上市櫃',
 '苗栗縣',
 '出口值']

In [ ]:
!pip install summa -q

In [ ]:
from summa import keywords

In [ ]:
keywords.keywords(text)

'明年\n純益\n市場\n表示 中國\n台灣\n需求\n台股\n挹注 營收\n成長\n大廠\n年增\n指數\n董事長\n客戶\n受惠\n可望\n第三 財報\n出貨\n影響\n隨著\n獲利\n持續\n全球\n產品 應用\n半導體 領域\n召開 總經理\n創下 新高\n疫情\n上市 公司\n公布 自結\n宣布\n族群 宇宙\n產業\n預計\nky\n認為\n投資\n預期\n國產 公告 高雄\n取得 美國\n聯發科\n資金\nic\n集團 旗下\n今年 比重 降至\n中心\n子公司\n第四 全數 滿載\n目前\n訂單\n單月\n布局\n計畫\n營運 副總 漢磊\n推升\n預估\n電子\n積極\n鴻海\n再度\n水準 之上 加上\n展望\n下滑\n旺季\n進入\n業者\n產能 屆時\n股價\n未來 事業\n去年 殖利率\n帶動\n車用\n合作\n技術\n投入 美元\n累計\n銀行 正式\n科技\n看好 電池\n日本\n同步\n包括\n毛利率 回升\n指出 今天\n疫苗\n單季 虧損\n優於\n價格 持穩 高檔 近期\n目標\n第一\n手機 晶片\n主要\n超過 漲幅\n美股\n維持 平盤\n筆電\n年底\n供應鏈 缺料\n執行\n台積電 先進 封裝 製程\n相關\n震盪\n資料\n元件\n成為\n大同\n設備\n醫療 擴大\n代工\n晶圓 代工廠\n品牌\n挑戰\n調整\n動能 延續\n本業\n終場 大漲 法人 合計 買超 外資\n開發\n高達\n整體\n景氣\n表現\n出爐\n調薪\n業務 發展 組織\n增加\n上漲\n服務\n電動車\n多頭\n國際 日前 傳出\n工業\n加速\n部分\n突破\n通路\n航空 雙雄 金融\n交易\n轉投資\n銷售 策略 執行長\n稅收\n生產\n因應\n金管會\n投資人\n創高\n經濟\n連續 入選\npcb\n規劃\n企業 處分 利益\n面板\n有望\n金額\n持股\n認證\n業績 明顯\n新台幣\n轉型\n大盤\n國內\n共同\n系統 建設\n伺服器\n大量\n新品 買盤\n完成\n機會 國銀\n黃天牧\n激勵\n進行\n第二 楊金龍\n通膨\n調漲\n衛星 題材 領軍\n強勁\n重回 大關\n歷史\n競爭\n至少\n推動\n平均\n貨櫃\n呈現 財政部\neps 歷年\n商機\n權值股\nmeta\n收斂\n首度\n資本\n電子股\

In [ ]:
! pip install textrank4zh -q

In [ ]:
from textrank4zh import TextRank4Keyword, TextRank4Sentence


In [ ]:
text = " ".join(org_corpus_df[org_corpus_df['summary_list_len']>2]['summary'][0:50])

In [ ]:
text

'混凝土廠國產 (2504-TW) 今 (15) 日公告，將高雄「悅誠廣場」房地持分以總價 23.5 億元，出售予興富發 (2542-TW) 子公司碧江企業，處分利益達 14.9 億元，估貢獻每股純益約 1.27 元。 台泥 (1101-TW) 今 (15) 日召開法說，總經理李鐘培表示，10 月以來中國能耗雙控持續，水泥產銷量減但價格持穩高檔，且近期煤價大幅回落，有利毛利率回升到過往水準之上，加上廣東韶關廠投產也將挹注營收表現。 台泥 (1101-TW) 董事長張安平今 (15) 日在法說會前致詞時指出，今天的氣候災害，是人類打造繁榮的副作用漸漸浮現、失衡的大自然對人類的反撲；與人類文明與大自然息息相關的水泥業「站在生命的第一現場」。 聚醯亞胺（PI）薄膜廠達邁 (3645-TW) 積極展開產品應用多元化的布局，由以往高達 98% 的軟板應用，在今年前三季的比重已降至 64%，而在非軟板的應用的成長到 36% 則以車用、散熱產品為主。 22 家壽險前 3 季合賺 3389 億元破紀錄 淨值比 all pass！ 康控 - KY(4943-TW) 今 (15) 日公告第三季財報，由於與銷貨客戶存在產品品質與責任歸屬認知上的差異，因此提列 26.3 億元預期信用減損，導致單季稅後虧損來到 27.38 億元，每股虧損 17.49 元。 台灣光罩 (2338-TW) 今 (15) 日公告，為因應光罩業務發展與組織擴展計畫，增設營運副總，由漢磊 (3707-TW) 前營運副總曾哲斌出任，由於漢磊長期著墨在化合物半導體領域，光罩此次派任具化合物半導體背景的經理人擔任要職 2021 道瓊永續指數 DJSI 評比出爐，東元連續兩年入選道瓊永續指數（DJSI）的新興市場指數 (DJSI-Emerging Markets)，為新興市場指數類別中，全球唯一入選的機電業者。 台泥 (1101-TW) 今 (15) 日召開法說，對於外界關注的綠能、儲能布局，董事長張安平也也對外揭櫫，儲能子公司 NHOA 的十倍成長的「10X 計畫」，預計 2030 在南歐充電樁市場達到 15% 市佔率，營收將會是 2022 年的 15 倍 檢測試劑廠瑞基 (4171-TW) 今 (15) 日表示，因應未來事業方向，進行內部組織升級，明年將朝三大方向發展，包括人病新冠檢測、畜產與伴侶動物，以及精準醫療，藉此

In [ ]:
tr4w = TextRank4Keyword()

tr4w.analyze(text=text, lower=True, window=2)  

print( '关键词：' )
for item in tr4w.get_keywords(20, word_min_len=1):
    print(item.word, item.weight)

print()
print( '关键短语：' )
for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num= 2):
    print(phrase)

tr4s = TextRank4Sentence()
tr4s.analyze(text=text, lower=True, source = 'all_filters')

print()
print( '摘要：' )
for item in tr4s.get_key_sentences(num=3):
    print(item.index, item.weight, item.sentence)

关键词：
後 0.007108566641203301
tw 0.006354432159484916
指數 0.005986742498391665
表示 0.005917911101960113
產業 0.005702043037101963
台灣 0.005240160746367019
今 0.0050560683721174865
行情 0.005002899380426773
股 0.004873901840539684
明年 0.004695063686162665
半導體 0.004597946079436361
銀行 0.004546633136017713
成長 0.00441836024074079
業務 0.004265133937898047
於 0.003917076665587562
服務 0.003910698231172071
客戶 0.0038247695511415183
董事 0.0035960976257943603
營收 0.0034748619364222497
持續 0.00335336095026319

关键短语：
股今

摘要：
24 0.053664578893588485 國賓飯店 (2704-TW) 的台北館及高雄館將分別投入危老改建，而欣陸 (3703-TW) 旗下大陸建設與國賓及厚生 (2107-TW) 今 (15) 日正式簽約，將就高雄國賓大飯店現址購入部分土地，並簽約後續的申請危老合建開發案， 航空客運市場復甦 台灣虎航開放冬季班表 4 航線 星宇航推曙光包機 玉山金控連續 8 年入選「道瓊永續指數」 再創台灣金融業紀錄 三引擎發威 台新人壽總投報率逾 3% 創高 未來續增海外債 「台灣友善環境指數」統計近 3 年來累計報酬率達 110.38%、去年年殖利率 4.9%，表現優於證交所發行的加權報酬指數 (89.29%、4.32%) 與台灣全市場報酬指數 (97.84 規劃以特別股與交換債 明年上半年續處分彰銀 家樂福與 Meta 策略合作推動數位轉型 9 大據點國家台灣入列 中鋼 (2002-TW) 今 (15) 日開出 12 月內銷鋼品月盤，由於中國寶武鋼以及越南河靜等鋼廠盤價走跌，衝擊亞洲行情及市場信心，國內產業也持續面臨庫存去化等壓力，經考量後，中鋼決議 12 月盤價平均調降 1.65%，近 

In [ ]:
text= """
文本關鍵詞抽取、文本摘要生成是自然語言處理（NLP）的應用之一，一定會對我們的生活產生巨大影響。隨着數字媒體的發展和出版業的不斷增長，誰還會有時間完整地瀏覽整篇文章、文檔、書籍來決定它們是否有用呢？值得高興的是，這項技術已經在這裏了。也就是今天我們要學習的TextRank算法。
   文本摘要可以大致分爲兩類——抽取型摘要 和 抽象型摘要：
   抽取型摘要： 這種方法依賴於從文本中提取幾個部分，例如短語、句子，把它們堆疊起來創建摘要。因此，這種抽取型的方法最重要的是識別出適合總結文本的句子。顯然，TextRank算法進行摘要抽取屬於抽取式的。
   抽象型摘要：這種方法應用先進的NLP技術生成一篇全新的總結。可能總結出的文本沒有在原文中出現。
目前主要方法有：
基於統計：統計詞頻，位置等信息，計算句子權值，再簡選取權值高的句子作爲文摘，特點：簡單易用，但對詞句的使用大多僅停留在表面信息。
基於圖模型：構建拓撲結構圖，對詞句進行排序。例如，TextRank/LexRank
基於潛在語義：使用主題模型，挖掘詞句隱藏信息。例如，採用LDA，HMM
基於整數規劃：將文摘問題轉爲整數線性規劃，求全局最優解。
一、算法思想
   TextRank算法是由PageRank算法改進而來的，二者的思想有相同之處，區別在於：PageRank算法根據網頁之間的鏈接關係構造網絡，而TextRank算法根據詞之間的共現關係構造網絡；PageRank算法構造的網絡中的邊是有向無權邊，而TextRank算法構造的網絡中的邊是無向有權邊。

   TextRank 算法是一種用於文本的基於圖的排序算法，通過把文本分割成若干組成單元（句子/詞），構建節點連接圖，用句子(詞)之間的相似度作爲邊的權重，通過循環迭代計算句子（詞）的TextRank值，最後抽取排名高的句子（詞）組合成文本摘要。
   上一篇博客我們已經學習了PageRank算法的原理。還未搞清楚的可以移步這裏

   用TextRank提取來提取關鍵詞/摘要，用PageRank的思想來解釋它：

如果一個單詞出現在很多單詞後面的話，那麼說明這個單詞比較重要
一個TextRank值很高的單詞後面跟着的單詞，那麼這個單詞的TextRank值會相應地因此而提高。
"""

r4w = TextRank4Keyword()

tr4w.analyze(text=text, lower=True, window=2)  

print( '关键词：' )
for item in tr4w.get_keywords(20, word_min_len=1):
    print(item.word, item.weight)

print()
print( '关键短语：' )
for phrase in tr4w.get_keyphrases(keywords_num=20, min_occur_num= 2):
    print(phrase)

tr4s = TextRank4Sentence()
tr4s.analyze(text=text, lower=True, source = 'all_filters')

print()
print( '摘要：' )
for item in tr4s.get_key_sentences(num=3):
    print(item.index, item.weight, item.sentence)

关键词：
句子 0.032153034889580996
文本 0.026132365502265626
textrank 0.02469339438859447
於 0.023027422824444834
算法 0.0202640798121058
摘要 0.020220638581788665
來 0.019324040142460783
抽取 0.017370316518096663
模型 0.012836513010605644
信息 0.012800822530012431
構建 0.012369235263166203
pagerank 0.012182839288405429
lda 0.011433937376233676
求 0.011433937376233676
關 0.011433937376233676
單詞 0.011208564480522482
nlp 0.010475328232796234
使用 0.010180844598599225
媒體 0.010178327040651346
發展 0.010178327040651346

关键短语：
文本摘要

摘要：
20 0.07550273771351866 TextRank 算法是一種用於文本的基於圖的排序算法，通過把文本分割成若干組成單元（句子/詞），構建節點連接圖，用句子(詞)之間的相似度作爲邊的權重，通過循環迭代計算句子（詞）的TextRank值，最後抽取排名高的句子（詞）組合成文本摘要
7 0.056575707432853437 顯然，TextRank算法進行摘要抽取屬於抽取式的
5 0.05420933232125146 抽取型摘要： 這種方法依賴於從文本中提取幾個部分，例如短語、句子，把它們堆疊起來創建摘要


## 結論

TF-IDF速度快，適合每一篇新聞的「摘要」綜合起來看，做出每一篇文章的關鍵字以及整體的關鍵字

textrank適合「單篇完整文章」抽取關鍵字和關鍵短句

https://blog.csdn.net/liujh845633242/article/details/103514778#2_191

## LDA主題模型

In [ ]:
words_ls = org_corpus_df[org_corpus_df['summary_list_len']>0]['summary_seg'][0:5000]

In [ ]:
from gensim import corpora, models

# 構造詞典
dictionary = corpora.Dictionary(words_ls)
# 基於詞典，使【詞】→【稀疏向量】，並將向量放入列表，形成【稀疏向量集】
corpus = [dictionary.doc2bow(words) for words in words_ls]
# lda模型，num_topics設定主題的個數
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=8)
# 印所有主題，每個主題顯示3個詞


In [ ]:
for topic in lda.print_topics(num_words=5):
    print(topic)

(0, '0.029*"純益" + 0.010*"獲利" + 0.010*"公布" + 0.009*"財報" + 0.009*"新高"')
(1, '0.007*"美國" + 0.006*"中國" + 0.006*"疫情" + 0.006*"開發" + 0.006*"台灣"')
(2, '0.009*"董事長" + 0.008*"表示" + 0.007*"市場" + 0.006*"公司" + 0.005*"營運"')
(3, '0.013*"需求" + 0.012*"營收" + 0.012*"表示" + 0.011*"召開" + 0.010*"成長"')
(4, '0.020*"台股" + 0.011*"指數" + 0.008*"台灣" + 0.008*"表示" + 0.006*"外資"')
(5, '0.044*"None" + 0.042*"營收" + 0.024*"年增" + 0.011*"公告" + 0.010*"新高"')
(6, '0.013*"台股" + 0.010*"台積電" + 0.009*"晶圓" + 0.007*"指數" + 0.006*"賣壓"')
(7, '0.034*"純益" + 0.010*"年增" + 0.009*"財報" + 0.007*"公告" + 0.005*"指數"')


## Word Embedding

- [中研院詞向量語料庫申請](http://ckipsvr.iis.sinica.edu.tw/cemb/reg.php)

In [ ]:
!gdown https://drive.google.com/uc?id=1kv0lAGnFQAc-2KceoyU85MH30ZXc8XF_

Downloading...
From: https://drive.google.com/uc?id=1kv0lAGnFQAc-2KceoyU85MH30ZXc8XF_
To: /content/CWE01.zip
100% 587M/587M [00:03<00:00, 147MB/s]


In [ ]:
!unzip CWE01.zip

Archive:  CWE01.zip
  inflating: w2v_CNA_ASBC_300d.vec   


In [ ]:
from gensim import models

word2vec_path = '/content/w2v_CNA_ASBC_300d.vec'
w2v_model = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=False,unicode_errors='ignore')

#limits

In [ ]:
len(w2v_model.vocab)

517015

In [ ]:
w2v_model[['台灣', '好棒']]

array([[ 1.164120e-01,  3.757080e-01,  2.316170e-01, -9.952500e-02,
        -1.714380e-01,  4.239700e-02, -1.805960e-01, -4.835500e-02,
        -1.943620e-01,  7.823000e-03, -2.563700e-01,  7.074400e-02,
         4.541900e-02,  1.970810e-01,  1.165900e-02, -2.238280e-01,
        -7.379000e-02, -8.486000e-02,  1.535190e-01, -2.355500e-02,
        -9.365500e-02,  1.818900e-01,  3.544020e-01,  2.216340e-01,
         1.524800e-02,  3.037800e-01, -3.261400e-01, -5.784900e-02,
        -6.522000e-03,  8.872200e-02, -1.997200e-02,  7.007600e-02,
         3.218800e-02,  1.473870e-01,  1.583200e-02,  1.424320e-01,
         1.154520e-01,  1.468230e-01,  3.285200e-02,  1.702130e-01,
        -7.904000e-03, -7.723800e-02, -1.560090e-01,  1.093360e-01,
         1.484830e-01,  1.946000e-02, -1.401810e-01,  1.094030e-01,
         2.937400e-02,  3.630950e-01, -6.418300e-02,  4.438100e-02,
        -1.212520e-01, -1.957500e-02,  2.369220e-01, -1.926540e-01,
         2.459730e-01,  2.470800e-02, -2.769900e

- `.bin` use: load_fasttext_format() (this typically contains full model with parameters, ngrams, etc).

- `.vec` use: load_word2vec_format (this contains ONLY word-vectors -> no ngrams + you can't update an model).

### Fasttext

In [ ]:
!gdown https://drive.google.com/uc?id=1wRvgAdpQ2roHPsbfrO8noUgoC54W_bGu

Downloading...
From: https://drive.google.com/uc?id=1wRvgAdpQ2roHPsbfrO8noUgoC54W_bGu
To: /content/cc.zh.300.bin
100% 7.24G/7.24G [01:41<00:00, 71.0MB/s]


In [ ]:
word2vec_path = '/content/cc.zh.300.bin'
from gensim.models import FastText

model = FastText.load_fasttext_format(word2vec_path)

In [ ]:
model[['你', '在']]

### 英文財金詞彙Embedding

In [ ]:
! wget https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip
!unzip /content/FinText_FastText_Skip-gram.zip


In [ ]:
from gensim.models import Word2Vec
from gensim.models import FastText


In [ ]:
prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [ ]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_FastText_Skip-gram/Word_Embedding_2000_2015')


In [ ]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]


## TRANSFORMER BERT

### 叫出最後一層的向量

In [ ]:
from transformers import AutoModel, AutoTokenizer

## 載入Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

## 載入Pretrain模型
model = AutoModel.from_pretrained('bert-base-chinese')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

## 獨立對待
sentences = [
  '基金類別生技股',
  '基金類別全球股市',
  '基金類別新興市場債',
  '基金類別公司債',
  '基金類別科技股',
  '基金類別全球債',
  '基金類別美國公債',
  '美國公債',
  '人情債怎麼還',
  '漲跌幅震盪',
  '均線黃金交叉',
  '股市大多頭',
  '小狗','小貓','人類']


context1 = ['暴雷一時爽', '醒醒吧']
context2 = ['一直暴雷一直爽', '你沒有妹妹']

In [ ]:
tokenizer.decode(tokenizer(sentences)['input_ids'][0])

'[CLS] 基 金 類 別 生 技 股 [SEP]'

In [ ]:
## 可用於問答or推論任務
tokenizer.decode(tokenizer(context1, context2)['input_ids'][0])

'[CLS] 暴 雷 一 時 爽 [SEP] 一 直 暴 雷 一 直 爽 [SEP]'

In [ ]:
tokenizer.decode(tokenizer(context1, context2)['input_ids'][1])

'[CLS] 醒 醒 吧 [SEP] 你 沒 有 妹 妹 [SEP]'

In [ ]:
## return_tensors="pt" 才能放進model
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
inputs

{'input_ids': tensor([[ 101, 1825, 7032, 7546, 1162, 4495, 2825, 5500,  102,    0,    0],
        [ 101, 1825, 7032, 7546, 1162, 1059, 4413, 5500, 2356,  102,    0],
        [ 101, 1825, 7032, 7546, 1162, 3173, 5646, 2356, 1842, 1002,  102],
        [ 101, 1825, 7032, 7546, 1162, 1062, 1385, 1002,  102,    0,    0],
        [ 101, 1825, 7032, 7546, 1162, 4906, 2825, 5500,  102,    0,    0],
        [ 101, 1825, 7032, 7546, 1162, 1059, 4413, 1002,  102,    0,    0],
        [ 101, 1825, 7032, 7546, 1162, 5401, 1751, 1062, 1002,  102,    0],
        [ 101, 5401, 1751, 1062, 1002,  102,    0,    0,    0,    0,    0],
        [ 101,  782, 2658, 1002, 2582, 7938, 6917,  102,    0,    0,    0],
        [ 101, 4039, 6649, 2388, 7448, 4679,  102,    0,    0,    0,    0],
        [ 101, 1772, 5221, 7941, 7032,  769, 1349,  102,    0,    0,    0],
        [ 101, 5500, 2356, 1920, 1914, 7531,  102,    0,    0,    0,    0],
        [ 101, 2207, 4318,  102,    0,    0,    0,    0,    0,    0,    0]

In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
x = outputs['last_hidden_state']

In [ ]:
embedding = x[:,-5:-1].mean(axis=1).cpu().detach().numpy()

In [ ]:
embedding.shape

(15, 768)

### 摘要

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

conversation = '''
Think of ETFs as buckets that hold a collection of securities, like stocks and bonds. Because ETFs are made up of these multiple assets, they provide investors instant diversification. When an investor purchases a share of an ETF, their money is spread across different investments. This differs from stocks where you buy shares of just a single company.

ETFs typically mimic a market index like the S&P 500. Since ETF performance is usually based on an index — meaning they follow the ups and downs of said index — most are passively managed investments and thus likely have lower fees than mutual funds. Mutual funds, on the other hand, want to beat the market’s performance and are thus managed by a fund manager, who’s actively choosing the investments.

Similar to stocks, ETFs can be bought and sold on an exchange throughout the day, and investors can even earn dividends depending on the type of index the fund tracks.

Should you invest in ETFs?
Since ETFs offer built-in diversification and don’t require large amounts of capital in order to invest in a range of stocks, they are a good way to get started. You can trade them like stocks while also enjoying a diversified portfolio.
'''



Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'summary_text': 'ETFs are buckets that hold a collection of securities, like stocks and bonds. They are passively managed and have lower fees than mutual funds. Similar to stocks, they can be bought and sold on an exchange throughout the day and investors can earn dividends depending on the type of index the fund tracks.'}]

In [ ]:
sumry = summarizer(conversation)

In [ ]:
sumry[0]['summary_text'].split('.')

['ETFs are buckets that hold a collection of securities, like stocks and bonds',
 ' They are passively managed and have lower fees than mutual funds',
 ' Similar to stocks, they can be bought and sold on an exchange throughout the day and investors can earn dividends depending on the type of index the fund tracks',
 '']

### 財金新聞情緒判別

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert",num_labels=3,cache_dir=None)

In [ ]:
sentences = ['Ruukki Group calculates that it has lost EUR 4mn in the failed project    ', 
             'it is ok and normal as usual',
             'sp500 surge soar',
             'Marathon estimates the value of its remaining stake in Protalix at $ 27 million']

In [ ]:
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## pos neg natral

model(**inputs)

SequenceClassifierOutput([('logits', tensor([[-0.8927,  2.4818, -1.8230],
                                   [-0.1370, -1.5946,  1.8591],
                                   [ 0.9483, -1.4564,  0.0273],
                                   [-0.9384, -1.1353,  2.6071]], grad_fn=<AddmmBackward0>))])